# <center> <span style="color: #7f1cdb;"><b>Tensor networks exercises</b></span>

## <span style="color: #e6023e;"><b>Library</b></span>

In [1]:
import numpy as np

from numpy import linalg as LA
from ncon import ncon

## <span style="color: #e6023e;"><b>Matrix Product States in open boundary conditions</b></span>

### <span style="color: #3b23ff;"><b>Exercise 1</b></span>

Write a function that creates a MPS with open boundary conditions where all the tensors-elements are 0.

The function will be called `zeros_mps()` and will have 3 input parameters:
- d: the physical or local dimension (it is the dimension of the leg pointing downwards).
- chi: the maximum bond dimension.
- n_sites: the number of sites of the lattice.

As output it will give a list with n_sites elements where every element is a 3 leg tensor full of zero's.

Hints:
- In an MPS with *open boundary conditions* not all the bonds (links between tensors) have the same dimension. 
Example: an MPS for a system of 10 qubits would have 9 links. If the maximum bond dimension is 14, the dimensions of these bonds would be 2, 4, 8, 14, 14, 14, 8, 4, 2.

- Use the function `np.zeros()`


In [5]:
# Exercise 1)

def zeros_mps(d, chi, n_sites):
    """
    Creates a Matrix Product State (MPS) with open boundary conditions where all tensor elements are zeros.

    Parameters:
    - d (int): The physical or local dimension (dimension of the leg pointing downwards).
    - chi (int): The maximum bond dimension.
    - n_sites (int): The number of sites in the lattice.

    Returns:
    - list of np.ndarray: A list containing `n_sites` tensors. Each tensor is a 3-leg tensor filled with zeros.
    """
    tensor_network = [0 for _ in range(n_sites)]
    mid_point = (n_sites // 2)

    for i in range(n_sites):
        if i <= mid_point:
            # Growing towards the middle
            left_dim = min(2**i, chi)
            right_dim = min(2**(i + 1), chi)
        else:
            # Symmetrically shrinking after the middle
            left_dim = tensor_network[n_sites - i - 1].shape[2]
            right_dim = tensor_network[n_sites - i - 1].shape[0]
        
        tensor_network[i] = np.zeros((left_dim, d, right_dim))

    return tensor_network

# Example usage:
ex1 = zeros_mps(2, 14, 8)
for tensor in ex1:
    print(tensor.shape)

(1, 2, 2)
(2, 2, 4)
(4, 2, 8)
(8, 2, 14)
(14, 2, 14)
(8, 2, 4)
(4, 2, 2)
(2, 2, 1)


### <span style="color: #3b23ff;"><b>Exercise 2</b></span>

a) Write one function `random_mps(d, chi, n_sites)` that gives as output an MPS with all its tensor-elements random complex numbers.

b) Write one function `product_mps(d, chi, n_sites)` that creates the MPS associated to the quantum state |0000...00>.

Hint: 
- Use the function `np.random.rand()`
- The imaginary unit in Python is `1j`

In [6]:
# Exercise 2.a)

def random_mps(d, chi, n_sites):
    """
    Creates a Matrix Product State (MPS) with open boundary conditions where all tensor elements are random complex numbers.

    Parameters:
    - d (int): The physical or local dimension (dimension of the leg pointing downwards).
    - chi (int): The maximum bond dimension.
    - n_sites (int): The number of sites in the lattice.

    Returns:
    - list of np.ndarray: A list containing `n_sites` tensors with random complex elements.
    """
    tensor_network = [0 for _ in range(n_sites)]
    mid_point = (n_sites // 2)

    for i in range(n_sites):
        if i <= mid_point:
            # Growing towards the middle
            left_dim = min(2**i, chi)
            right_dim = min(2**(i + 1), chi)
        else:
            # Symmetrically shrinking after the middle
            left_dim = tensor_network[n_sites - i - 1].shape[2]
            right_dim = tensor_network[n_sites - i - 1].shape[0]
        
        tensor_network[i] = np.random.rand(left_dim, d, right_dim) + (1j * np.random.rand(left_dim, d, right_dim))

    return tensor_network

# Example usage:
ex2a = random_mps(2, 14, 8)
print(ex2a[0])

[[[0.94566945+0.20574864j 0.21343418+0.29358481j]
  [0.18680169+0.52502197j 0.14840507+0.273702j  ]]]


In [16]:
# Exercise 2.b)

def product_mps(d, chi, n_sites):
    """
    Creates the Matrix Product State (MPS) associated with the quantum state |000...00> with open boundary conditions.

    Parameters:
    - d (int): The physical or local dimension (dimension of the leg pointing downwards).
    - chi (int): The maximum bond dimension.
    - n_sites (int): The number of sites in the lattice.

    Returns:
    - list of np.ndarray: A list containing `n_sites` tensors representing the product state |000...00>.
    """
    tensor_network = []
    mid_point = n_sites // 2

    # Precompute bond dimensions ensuring symmetry
    bond_dims = [1]  # Starting with bond dimension 1 at the left boundary
    for i in range(1, mid_point + 1):
        bond_dim = min(2**i, chi)
        bond_dims.append(bond_dim)

    # For even n_sites, avoid duplicating the midpoint bond dimension
    if n_sites % 2 == 0:
        for i in range(mid_point - 1, 0, -1):
            bond_dim = min(2**i, chi)
            bond_dims.append(bond_dim)
    else:
        # For odd n_sites, include the midpoint bond dimension once
        for i in range(mid_point, 0, -1):
            bond_dim = min(2**i, chi)
            bond_dims.append(bond_dim)

    bond_dims.append(1)  # Ending with bond dimension 1 at the right boundary

    # Ensure bond_dims has length n_sites + 1
    if len(bond_dims) != n_sites + 1:
        raise ValueError(f"Incorrect number of bond dimensions: expected {n_sites +1}, got {len(bond_dims)}")

    for i in range(n_sites):
        left_dim = bond_dims[i]
        right_dim = bond_dims[i + 1]
        
        # Initialize the tensor with zeros and set the |0> state
        tensor = np.zeros((left_dim, d, right_dim), dtype=np.complex128)
        tensor[0, 0, 0] = 1.0  # Set the first element to 1 for |0> state (1,0)
        tensor_network.append(tensor)
    
    return tensor_network

# Example usage:
ex2b = product_mps(2, 14, 8)
for idx, tensor in enumerate(ex2b):
    print(f"Site {idx}: shape {tensor.shape}")
    print(tensor)


Site 0: shape (1, 2, 2)
[[[1.+0.j 0.+0.j]
  [0.+0.j 0.+0.j]]]
Site 1: shape (2, 2, 4)
[[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]]
Site 2: shape (4, 2, 8)
[[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]]
Site 3: shape (8, 2, 14)
[[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
   0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
   0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.

## <span style="color: #e6023e;"><b>Coefficient of a computational basis element</b></span>

### <span style="color: #3b23ff;"><b>Exercise 3</b></span>

a) Write a function that returns the value of the coefficient of a given element of the computational basis.

The element of the computational basis can be indicated by a bit string s of length n_sites. *Example*: If the bit string is '000101110', the outcome of the function is <000101110|MPS>

b) Test the  function `product_mps(d, chi, n_sites)` that you have created in the previous exercice and check that 
<000...00|Product state>=1 and 0 otherwise.

In [17]:
# Exercise 3.a)

def coeff_mps(s, mps):
    """
    Computes the coefficient <s|MPS> where |s> is a computational basis state represented by a bit string.

    Parameters:
    - s (str): Bit string representing the computational basis state (e.g., '000101110').
    - mps (list of np.ndarray): List of MPS tensors with open boundary conditions.

    Returns:
    - complex: The coefficient <s|MPS>.
    """
    if not isinstance(s, str):
        raise ValueError("The computational basis state 's' must be a string of bits.")
    
    n_sites = len(mps)
    if len(s) != n_sites:
        raise ValueError(f"The length of 's' ({len(s)}) must match the number of sites in MPS ({n_sites}).")
    
    # Initialize the contraction to a 1-dimensional array with value 1
    result = np.array([1.0 + 0.0j])
    
    for i, bit in enumerate(s):
        try:
            s_i = int(bit)
        except ValueError:
            raise ValueError(f"Invalid character '{bit}' in bit string. Must be '0' or '1'.")
        
        tensor = mps[i]
        left_dim, d, right_dim = tensor.shape
        
        if s_i >= d:
            # The physical index s_i is out of bounds for this tensor
            return 0.0 + 0.0j
        
        # Select the slice of the tensor corresponding to the bit s_i
        tensor_slice = tensor[:, s_i, :]  # Shape: (left_dim, right_dim)
        
        # Perform the contraction
        if result.shape[0] != left_dim:
            raise ValueError(f"Mismatch in bond dimensions at site {i}: expected {left_dim}, got {result.shape[0]}")
        
        result = np.dot(result, tensor_slice)  # Shape: (right_dim,)
    
    return result[0]


In [18]:
# Exercise 3.b)

def test_coeff_product_mps():
    """
    Tests the `coeff_mps` function using the `product_mps`.
    Verifies that <000...0|Product MPS> = 1 and <s|Product MPS> = 0 for s ≠ 000...0.
    """
    d = 2
    chi = 14
    n_sites = 8
    mps = product_mps(d, chi, n_sites)
    
    # Test Case 1: All bits are '0'
    s_zero = '0' * n_sites
    coeff_zero = coeff_mps(s_zero, mps)
    print(f"<{s_zero}|Product MPS> = {coeff_zero}")
    assert np.isclose(coeff_zero, 1.0), "Test Case 1 Failed: <000...0|Product MPS> should be 1."
    
    # Test Case 2: One bit is '1' at the end
    s_other = '0' * (n_sites - 1) + '1'
    coeff_other = coeff_mps(s_other, mps)
    print(f"<{s_other}|Product MPS> = {coeff_other}")
    assert np.isclose(coeff_other, 0.0), "Test Case 2 Failed: <s|Product MPS> should be 0 for s ≠ 000...0."
    
    # Test Case 3: One bit is '1' at the beginning
    s_other2 = '1' + '0' * (n_sites - 1)
    coeff_other2 = coeff_mps(s_other2, mps)
    print(f"<{s_other2}|Product MPS> = {coeff_other2}")
    assert np.isclose(coeff_other2, 0.0), "Test Case 3 Failed: <s|Product MPS> should be 0 for s ≠ 000...0."
    
    # Test Case 4: Arbitrary bit string
    s_other3 = '01010101'
    coeff_other3 = coeff_mps(s_other3, mps)
    print(f"<{s_other3}|Product MPS> = {coeff_other3}")
    assert np.isclose(coeff_other3, 0.0), "Test Case 4 Failed: <s|Product MPS> should be 0 for s ≠ 000...0."
    
    print("All tests passed successfully!")

test_coeff_product_mps()


<00000000|Product MPS> = (1+0j)
<00000001|Product MPS> = 0j
<10000000|Product MPS> = 0j
<01010101|Product MPS> = 0j
All tests passed successfully!


## <span style="color: #e6023e;"><b>Left orthogonal form of and norm</b></span>

### <span style="color: #3b23ff;"><b>Exercise 4</b></span>

The left orhtogonal form is defined as follows

<p style="text-align: center"><img src="https://imgur.com/ZraBWnX.png" width=700 /></p>

with the property that

<p style="text-align: center"><img src="https://imgur.com/vg1ig1j.png" width=250 /></p>

a) Write a function that brings a single tensor of an MPS in a *left orthogonal form*. The input of the function is a 3-leg tensor and the output is the new 3-leg tensor in its left-orthogonal form and a matrix that needs to plugged into the left leg of the right neighbour tensor in the MPS in order that the global MPS does not change.

b) Write a function that brings the first *n_stop* tensors of an MPS (starting from the left) into a *left orthogonal form*. If no *n_stop* is provided, it brings all the tensors of the MPS into the left orthogonal form, except for the last one (first tensor of the right).



In [ ]:
# Exercise 4.a)


In [ ]:
# Exercise 4.b)


### <span style="color: #3b23ff;"><b>Exercise 5</b></span>

a) Write a function `norm_mps()` that computes the norm of an MPS, that is, the contraction of an MPS with itself (complex conjugate).

b) Test the functions of the previous exercices *4.b* and *5.a* by:
- generating a random MPS *A* and computing its norm.

- bringing *A* into an left orthogonal form.

- computing the norm of *A* by only contracting its last tensor.

If the previous functions work, the norm of the original MPS needs to coincide with the norm of the last tensor of MPS in the left orthogonal form.

In [ ]:
# Exercise 5.a)


In [ ]:
# Exercise 5.b)


## <span style="color: #e6023e;"><b>Reduced density matrix of a single qubit</b></span>
### <span style="color: #3b23ff;"><b>Exercise 6</b></span>

Write a function that computes the **the reduced density matrix** of a single qubit of an MPS. 

The function will have as input the list of tensors that represent the MPS and the position of a qubit for which the density matrix needs to be calculated. As output it will give the reduced density matrix of the indicated qubit, a  2x2 matrix.

Hints:

- Diagramatically, the reduced density matrix of a qubit is very similar to the norm of the MPS. The difference is that while in the norm of the MPS we have 2 copies of the MPS with all the legs between them contracted, the reduced density matrix has all the legs contracted except for the two corresponding to the qubit whose reduced density matrix is computed.

- It can be useful to write first two functions that compute the left environment and the right environment of the qubit.

In [ ]:
# Exercise 6)



## <span style="color: #e6023e;"><b>Center of orthogonality</b></span>
### <span style="color: #3b23ff;"><b>Exercise 7</b></span>

Implement the center of orthogonality in the link located between tensors B and C.


<p style="text-align: center"><img src="https://imgur.com/gICcVuK.png" width=500 /></p>

The new netwoks tensor should look like the following image

<p style="text-align: center"><img src="https://imgur.com/nnyRNQQ.png" width=500 /></p>

In order to convert the link into an orthogonality center, the following steps are necessary:

- Calculate the density matrix to the left and to the right of the link, $\rho_{left}$ and $\rho_{right}$. 
- Find the matrices $X$ and $X^{\dagger}$ such that $\rho= XX^{\dagger}$ for each density matrix.
- Calculate  $\sigma'$ as $\sigma' = X_{left} I X_{right}$
- Calculate  $B'$ as $B' = BX_{left}^{-1}$
- Calculate  $C'$ as $C' = X_{right}^{-1}C$

<span style="color: #FFA500;">If you are blocked</span> <span style="color: yellow;">[click here](https://www.tensors.net/tutorial-4)</span>

In [ ]:
# Exercise 7)
